In [4]:
import sys
sys.path.append('../src')
import ed_function as eda

# Load data
df = eda.load_data('../data/raw/raw_analyst_ratings.csv')
df.head()

# Add headline length
df = eda.add_headline_length(df)
eda.describe_headline_length(df)

count    1.407328e+06
mean     7.312051e+01
std      4.073531e+01
min      3.000000e+00
25%      4.700000e+01
50%      6.400000e+01
75%      8.700000e+01
max      5.120000e+02
Name: headline_length, dtype: float64